In [ ]:
import os
import uuid

import boto3
from dotenv import load_dotenv

from bring_a_crew_bedrock.inline_agent_utils import invoke_inline_agent_helper, load_json_file
from bring_a_crew_bedrock.lambda_creator import create_lambda_function_and_its_resources, \
    remove_lambda_function_and_its_resources

_ = load_dotenv()

region = "eu-west-1"
model = "eu.amazon.nova-lite-v1:0"
# model = "eu.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Runtime Endpoints
bedrock_rt_client = boto3.client(
    "bedrock-agent-runtime",
    region_name=region
)

sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]


## Initialize the lambdas
The lambdas will be created or loaded if they already exist.

In [ ]:
present_directory = os.getcwd()


def register_lambda(lambda_name_part: str):
    lambda_function_code_path = str(present_directory) + f"/lambdas/lambda_function_{lambda_name_part}.py"

    # Create all resources
    resources = create_lambda_function_and_its_resources(
        region=region,
        account_id=account_id,
        custom_name=f"inlineagent-{lambda_name_part}",
        lambda_code_path=lambda_function_code_path
    )

    # Access the created resources
    lambda_function = resources['lambda_function']
    return lambda_function['FunctionArn']


lambda_schedule_arn = register_lambda("schedule")
lambda_facility_arn = register_lambda("facility")
lambda_caterer_arn = register_lambda("caterer")

## Initialize the collaborator agents
The collaborator agents will be created with the lambda functions and the instructions.

* `Schedule Agent` - This agent is responsible for scheduling the appointments with persons.
* `Facility Manager Agent` - This agent is responsible for checking the availability of rooms and booking them.
* `Caterer Agent` - This agent is responsible for ordering food and drinks.

In [ ]:
payload_schedule_path = str(present_directory) + "/lambdas/payload-schedule.json"
fetch_schedule_payload = load_json_file(payload_schedule_path)

schedule_request_params = {
    "agentName": "schedule_agent",
    "foundationModel": model,
    "instruction": (
        "You are a scheduling assistant. You help in checking the availability of people. Do not make up "
        "availability if you do not know the person's schedule. Answer in short sentences, stick to the "
        "answer to the question. With the availability of people you can schedule a meeting, if no specific "
        "date is asked, choose the first moment all participants are available. "
    ),
    "agentCollaboration": "DISABLED",
    "actionGroups": [
        {
            "actionGroupName": "FetchSchedule",
            "actionGroupExecutor": {
                "lambda": lambda_schedule_arn,
            },
            "apiSchema": {
                "payload": fetch_schedule_payload
            },
            "description": "Find availability of a person"
        }
    ]
}

payload_facility_path = str(present_directory) + "/lambdas/payload-facility.json"
fetch_facility_payload = load_json_file(payload_facility_path)

facility_request_params = {
    "agentName": "facility_agent",
    "foundationModel": model,
    "instruction": (
        "You manage the available rooms. You can check for availability of a room and book it for the "
        "meeting. You need the amount of people in the meeting, the date of the meeting and the timeslot (morning or afternoon)."
    ),
    "agentCollaboration": "DISABLED",
    "actionGroups": [
        {
            "actionGroupName": "FetchFacility",
            "actionGroupExecutor": {
                "lambda": lambda_facility_arn
            },
            "apiSchema": {
                "payload": fetch_facility_payload
            },
            "description": "Find available room"
        }
    ]
}

payload_caterer_path = str(present_directory) + "/lambdas/payload-caterer.json"
fetch_caterer_payload = load_json_file(payload_caterer_path)

caterer_request_params = {
    "agentName": "caterer_agent",
    "foundationModel": model,
    "instruction": (
        "You order food for meetings. You need the amount of people for lunch as well as the meeting "
        "date and the room."
    ),
    "agentCollaboration": "DISABLED",
    "actionGroups": [
        {
            "actionGroupName": "FetchCaterer",
            "actionGroupExecutor": {
                "lambda": lambda_caterer_arn
            },
            "apiSchema": {
                "payload": fetch_caterer_payload
            },
            "description": "Find available food and drinks"
        }
    ]
}

## Initialise the supervisor agent

In [ ]:
supervisor_request_params = {
    "enableTrace": True,
    "endSession": False,
    "foundationModel": model,
    "instruction": (
        "You assist in organizing meetings. For each meeting you need:"
        "\n- The names of the people that join the meeting."
        "\n- The date after which the meeting takes place and the timeslot of the meeting (morning or afternoon)."
        "\n- You need to know the lunch that people want, if they need lunch."
        "\nYou have other agents at your disposal:"
        "\n- Start with the schedule agent to find the date for the meeting."
        "\n- Next, use the facility agent to find and book a room for the meeting. Use the meeting date and timeslot obtained through the schedule agent."
        "\n- Finally, use the caterer agent to order lunch for the meeting. Order lunch only for people in the meeting, you need the room information and the meeting date to order lunch."
        "\nYou can ask the user for more information if you are missing some information."
    ),
    "sessionId": str(uuid.uuid4()),
    "agentCollaboration": "SUPERVISOR",
    "collaboratorConfigurations": [
        {
            "collaboratorInstruction": "Use to find the first suitable date the requested people are together available. After the provided date.",
            "collaboratorName": "schedule_agent",
            "relayConversationHistory": "DISABLED"
        },
        {
            "collaboratorInstruction": "Use to find a room for the amount of people in the meeting and the exact date you found before.",
            "collaboratorName": "facility_agent",
            "relayConversationHistory": "DISABLED"
        },
        {
            "collaboratorInstruction": "Use to order lunch. You need the amount of people in the meeting and the room id. You can ask for lunch options if you want.",
            "collaboratorName": "caterer_agent",
            "relayConversationHistory": "DISABLED"
        }
    ],
    "collaborators": [
        schedule_request_params, caterer_request_params, facility_request_params
    ],
    "inputText": "Schedule a meeting with Bob and Charlie as soon as possible after 2025-04-07, order lunch for them as well."
}

In [ ]:
invoke_inline_agent_helper(bedrock_rt_client, supervisor_request_params, trace_level="core")

In [ ]:
supervisor_request_params["inputText"] = "Broodje kroket."
invoke_inline_agent_helper(bedrock_rt_client, supervisor_request_params, trace_level="core")

In [ ]:
# Warning only run this cell when you are done ...

def remove(lambda_name_part: str):
    remove_lambda_function_and_its_resources(
        region=region,
        account_id=account_id,
        custom_name=f"inlineagent-{lambda_name_part}",
    )

# remove("schedule")
# remove("facility")
# remove("caterer")
